# TSA_ch8_model_comparison

## Comprehensive Model Comparison: ARIMA vs ARFIMA vs Random Forest vs LSTM

**Data**: Germany Daily Electricity Consumption (2015-2019)

**Source**: ENTSO-E Transparency Platform / Open Power System Data

**Author**: Daniel Traian PELE

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# LSTM (optional)
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.callbacks import EarlyStopping
    TF_AVAILABLE = True
    tf.random.set_seed(42)
except ImportError:
    TF_AVAILABLE = False
    print("TensorFlow not available. LSTM will be skipped.")

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Load Germany Electricity Data

In [ ]:
# Download from Open Power System Data
url = "https://data.open-power-system-data.org/time_series/2020-10-06/time_series_60min_singleindex.csv"

print("Downloading Germany electricity data...")
df_raw = pd.read_csv(url, parse_dates=['utc_timestamp'], low_memory=False)

df = df_raw[['utc_timestamp', 'DE_load_actual_entsoe_transparency']].copy()
df.columns = ['datetime', 'load_mw']
df = df.dropna()

df['date'] = df['datetime'].dt.date
daily = df.groupby('date')['load_mw'].sum().reset_index()
daily['date'] = pd.to_datetime(daily['date'])
daily['consumption_gwh'] = daily['load_mw'] / 1000

daily = daily[(daily['date'] >= '2015-01-01') & (daily['date'] <= '2019-12-31')]
daily = daily.reset_index(drop=True)

print(f"Data: {daily['date'].min().date()} to {daily['date'].max().date()}")
print(f"Observations: {len(daily)} days")

## 2. Feature Engineering

In [ ]:
def create_features(df):
    df = df.copy()
    df['y'] = df['consumption_gwh']
    
    for lag in [1, 2, 3, 7, 14, 21]:
        df[f'lag_{lag}'] = df['y'].shift(lag)
    
    for window in [7, 14, 30]:
        df[f'roll_mean_{window}'] = df['y'].shift(1).rolling(window).mean()
        df[f'roll_std_{window}'] = df['y'].shift(1).rolling(window).std()
    
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    return df.dropna()

df_feat = create_features(daily)
feature_cols = [c for c in df_feat.columns if c not in ['date', 'load_mw', 'consumption_gwh', 'y']]

## 3. Train/Test Split

In [ ]:
n = len(df_feat)
train_end = int(n * 0.70)
val_end = int(n * 0.85)

train = df_feat.iloc[:train_end]
val = df_feat.iloc[train_end:val_end]
test = df_feat.iloc[val_end:]

print(f"Train: {len(train)} | Val: {len(val)} | Test: {len(test)}")

## 4. Model Training

In [ ]:
def mape(y_true, y_pred):
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

results = []
predictions = {}

In [ ]:
# 1. Naive Baseline
naive_pred = test['lag_1'].values
results.append({'Model': 'Naive (y_{t-1})', 'MAPE': mape(test['y'].values, naive_pred)})
predictions['Naive'] = naive_pred
print("[1/6] Naive: done")

# 2. Seasonal Naive
seasonal_pred = test['lag_7'].values
results.append({'Model': 'Seasonal Naive (y_{t-7})', 'MAPE': mape(test['y'].values, seasonal_pred)})
predictions['Seasonal'] = seasonal_pred
print("[2/6] Seasonal Naive: done")

In [ ]:
# 3. SARIMA (rolling forecast)
print("[3/6] SARIMA: training (rolling forecasts)...")
sarima_pred = []
history = list(train['y'].values)

for t in range(len(test)):
    try:
        model = ARIMA(history, order=(1, 1, 1))
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
    except:
        yhat = history[-1]
    sarima_pred.append(yhat)
    history.append(test['y'].iloc[t])
    if (t+1) % 50 == 0:
        print(f"    Progress: {t+1}/{len(test)}")

sarima_pred = np.array(sarima_pred)
results.append({'Model': 'SARIMA(1,1,1)', 'MAPE': mape(test['y'].values, sarima_pred)})
predictions['SARIMA'] = sarima_pred
print("       SARIMA: done")

In [ ]:
# 4. Random Forest
print("[4/6] Random Forest: training...")
rf = RandomForestRegressor(n_estimators=200, max_depth=15, min_samples_leaf=5, random_state=42, n_jobs=-1)
rf.fit(train[feature_cols], train['y'])
rf_pred = rf.predict(test[feature_cols])
results.append({'Model': 'Random Forest', 'MAPE': mape(test['y'].values, rf_pred)})
predictions['RF'] = rf_pred
print("       Random Forest: done")

In [ ]:
# 5. LSTM
if TF_AVAILABLE:
    print("[5/6] LSTM: training...")
    
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train[['y']])
    val_scaled = scaler.transform(val[['y']])
    test_scaled = scaler.transform(test[['y']])
    
    seq_len = 14
    
    def create_seq(data, seq_len):
        X, y = [], []
        for i in range(len(data) - seq_len):
            X.append(data[i:i+seq_len])
            y.append(data[i+seq_len])
        return np.array(X), np.array(y)
    
    X_train, y_train = create_seq(train_scaled.flatten(), seq_len)
    X_val, y_val = create_seq(np.concatenate([train_scaled[-seq_len:], val_scaled]).flatten(), seq_len)
    X_test, y_test_lstm = create_seq(np.concatenate([val_scaled[-seq_len:], test_scaled]).flatten(), seq_len)
    
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    model = Sequential([
        LSTM(64, activation='relu', return_sequences=True, input_shape=(seq_len, 1)),
        Dropout(0.2),
        LSTM(32, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, 
              callbacks=[early_stop], verbose=0)
    
    lstm_pred_scaled = model.predict(X_test, verbose=0)
    lstm_pred = scaler.inverse_transform(lstm_pred_scaled).flatten()
    
    y_test_actual = test['y'].values[-len(lstm_pred):]
    results.append({'Model': 'LSTM', 'MAPE': mape(y_test_actual, lstm_pred)})
    predictions['LSTM'] = lstm_pred
    print("       LSTM: done")
else:
    print("[5/6] LSTM: skipped (TensorFlow not available)")

## 5. Results Summary

In [ ]:
results_df = pd.DataFrame(results).sort_values('MAPE')

print("="*60)
print("MODEL COMPARISON - GERMANY ELECTRICITY FORECAST")
print("="*60)
print(f"\n{results_df.to_string(index=False)}")
print(f"\n*** Best Model: {results_df.iloc[0]['Model']} (MAPE = {results_df.iloc[0]['MAPE']:.2f}%) ***")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Predictions comparison
ax = axes[0, 0]
ax.plot(test['date'], test['y'], 'k-', linewidth=1.5, label='Actual')
ax.plot(test['date'], predictions['Seasonal'], '--', color='gray', alpha=0.6, label='Seasonal Naive')
ax.plot(test['date'], predictions['RF'], '-', color='green', linewidth=1, label='Random Forest')
ax.plot(test['date'], predictions['SARIMA'], '-', color='blue', alpha=0.6, linewidth=1, label='SARIMA')
if 'LSTM' in predictions:
    ax.plot(test['date'].values[-len(predictions['LSTM']):], predictions['LSTM'], 
            '-', color='red', linewidth=1, label='LSTM')
ax.set_title('Germany Electricity: Model Predictions', fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Consumption (GWh)')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. MAPE comparison
ax = axes[0, 1]
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(results_df)))
bars = ax.barh(results_df['Model'], results_df['MAPE'], color=colors)
ax.set_xlabel('MAPE (%)')
ax.set_title('Model Performance (Lower = Better)', fontweight='bold')
for bar, val in zip(bars, results_df['MAPE']):
    ax.text(val + 0.1, bar.get_y() + bar.get_height()/2, f'{val:.2f}%', va='center')
ax.grid(True, alpha=0.3, axis='x')

# 3. Feature importance
ax = axes[1, 0]
importance = pd.DataFrame({'feature': feature_cols, 'importance': rf.feature_importances_})
importance = importance.sort_values('importance', ascending=False).head(10)
ax.barh(importance['feature'], importance['importance'], color='forestgreen')
ax.invert_yaxis()
ax.set_xlabel('Importance')
ax.set_title('Random Forest: Top 10 Features', fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# 4. Last 60 days detail
ax = axes[1, 1]
ax.plot(test['date'].values[-60:], test['y'].values[-60:], 'ko-', markersize=3, label='Actual')
ax.plot(test['date'].values[-60:], predictions['RF'][-60:], 'g-', linewidth=1.5, label='RF')
ax.plot(test['date'].values[-60:], predictions['Seasonal'][-60:], '--', color='gray', label='Seasonal')
ax.set_title('Last 60 Days: Detailed View', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ch8_model_comparison.pdf', bbox_inches='tight', dpi=150)
plt.show()

## 6. Key Findings

1. **Random Forest** typically performs best for complex multi-seasonal patterns
2. **Seasonal Naive** (y_{t-7}) is a surprisingly strong baseline for weekly data
3. **Feature engineering** (lags, rolling stats, calendar features) is crucial
4. **LSTM** requires more data and tuning to outperform simpler methods
5. Model selection depends on data characteristics and forecast horizon